# Lesson 3: Reflection and Blogpost Writing

## Setup

In [1]:
import os
from dotenv import load_dotenv
llm_config = {
    "config_list": [{ 'model': 'gpt3516k',
        'api_key': ''+(os.environ.get('AZURE_OPENAI_KEY'))+'',
        'base_url':  ''+(os.environ.get('AZURE_OPENAI_ENDPOINT'))+'',
        'api_type': 'azure',
        'api_version': '2024-02-01'}]
        }

## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

DeepLearning.AI: Unlocking the Potential of Artificial Intelligence

DeepLearning.AI is revolutionizing the world of artificial intelligence. With its cutting-edge deep learning technologies, this organization is paving the way for unprecedented advancements in various industries. From self-driving cars to personalized medicine, DeepLearning.AI is empowering companies to harness the true potential of AI. With an expert team of researchers and engineers, they are at the forefront of AI innovation. Join the journey towards a smarter future with DeepLearning.AI and unlock the infinite possibilities of AI.


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

DeepLearning.AI: Unlocking the Potential of Artificial Intelligence

DeepLearning.AI is revolutionizing the world of artificial intelligence. With its cutting-edge deep learning technologies, this organization is paving the way for unprecedented advancements in various industries. From self-driving cars to personalized medicine, DeepLearning.AI is empowering companies to harness the true potential of AI. With an expert team of researchers and engineers, they are at the forefront of AI innovation. Join the journey towards a smarter future with DeepLearning.AI and unlock the infinite possibilities of AI.

--------------------------------------------------------------------------------
Critic (to Writer):

Title: DeepLearning.AI: Unl

## Nested chat

In [8]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [12]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [13]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [14]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

DeepLearning.AI: Unlocking the Potential of Artificial Intelligence

DeepLearning.AI is revolutionizing the world of artificial intelligence. With its cutting-edge deep learning technologies, this organization is paving the way for unprecedented advancements in various industries. From self-driving cars to personalized medicine, DeepLearning.AI is empowering companies to harness the true potential of AI. With an expert team of researchers and engineers, they are at the forefront of AI innovation. Join the journey towards a smarter future with DeepLearning.AI and unlock the infinite possibilities of AI.

--------------------------------------------------------------------------------

***********************************************

## Get the summary

In [15]:
print(res.summary)

DeepLearning.AI: Unlocking the Potential of Artificial Intelligence

DeepLearning.AI is at the forefront of the artificial intelligence revolution. With its cutting-edge deep learning technologies, this organization is driving incredible advancements in various industries. From self-driving cars to personalized medicine, DeepLearning.AI empowers companies with improved efficiency, cost savings, and enhanced capabilities. With an expert team of researchers and engineers, they are leading AI innovation. Join the journey towards a smarter future by unleashing the infinite possibilities of AI with DeepLearning.AI. Experience the benefits and be part of the AI transformation.
